* Time: Fall, 2019 
* Practice: STP 

Created to QuantEcon, Paralle Computing Repository [Practice 1](https://github.com/davidzarruk/Parallel_Computing/blob/master/Python_numba_main.py) and [Practice 2](https://github.com/davidzarruk/Parallel_Computing/blob/master/Python_main.py)

## House Keeping 

In [1]:
from numba import jit, jitclass, njit, prange, int64, float64
import numpy as np 
import math
import time
from scipy.stats import norm
from joblib import Parallel, delayed
import multiprocessing
from collections import OrderedDict
import sys

## Initialization 

In [2]:
# Number of workers 
num_cores = 5

# Grid for x
nx = 1500 
xmin = 0.1 
xmax = 4.0 

# Grid for e: parameters for Tauchen 
ne            = 15;
ssigma_eps    = 0.02058;
llambda_eps   = 0.99;
m             = 1.5;

# Utility function
ssigma        = 2;
bbeta         = 0.97;
T             = 10;

# Prices
r             = 0.07;
w             = 5;

In [3]:
# Initialize the grid for X 
xgrid = np.zeros(nx)

# Initialize the grid for E and the transition probability matrix 
egrid = np.zeros(ne)
P = np.zeros((ne, ne))

# Initialize value function V 
V = np.zeros((T, nx, ne))

# Grid Creation 

In [4]:
# Function to construct the grid for capital (x)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;

for i in range(0,ne):
	egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
	it = it+1;

# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
	for k in range(0,ne):
		if (k == 0):
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
		elif (k == ne-1):
			P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
		else:
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] +
            (mm/2))/ssigma_eps) - norm.cdf((egrid[k] 
            - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);


# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]); 

# Structure and Function 

In [5]:
# Value function
VV = math.pow(-10.0, 5);

# Data structure of state and exogenous variables
class modelState(object):
	def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r):
		self.ind		= ind
		self.ne			= ne
		self.nx			= nx
		self.T			= T
		self.age		= age
		self.P			= P
		self.xgrid		= xgrid
		self.egrid		= egrid
		self.ssigma		= ssigma
		self.bbeta		= bbeta
		self.w			= w
		self.r			= r

# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix)
def value_func(states):

	ind = states.ind
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r

	ix = int(math.floor(ind/ne));
	ie = int(math.floor(ind%ne));

	VV = math.pow(-10.0, 3)
	for ixp in range(0,nx):
		expected = 0.0;
		if(age < T-1):
			for iep in range(0,ne):
				expected = expected + P[ie, iep]*V[age+1, ixp, iep]

		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp];

		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;

		if(cons <= 0):
			utility = math.pow(-10.0,5);
		
		if(utility >= VV):
			VV = utility;

		utility = 0.0;

	return[VV];


## *Life-cycle Computation* 

In [6]:
print(" ")
print("Life cycle computation: ")
print(" ")


start = time.time()

for age in reversed(range(0,T)):

	# This function computes `value_func` in parallel for all the states
	results = Parallel(n_jobs=num_cores)(delayed(value_func)(modelState(ind,ne
                                                                        ,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r)) for ind in range(0,nx*ne))

	# I write the results on the value matrix: V
	for ind in range(0,nx*ne):
		
		ix = int(math.floor(ind/ne));
		ie = int(math.floor(ind%ne));

		V[age, ix, ie] = results[ind][0];

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")

finish = time.time() - start
print ("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")

 
Life cycle computation: 
 
Age:  10 . Time:  50.5793  seconds.
Age:  9 . Time:  608.3357  seconds.
Age:  8 . Time:  1126.0783  seconds.
Age:  7 . Time:  7504.9021  seconds.
Age:  6 . Time:  12252.161  seconds.
Age:  5 . Time:  16073.9248  seconds.
Age:  4 . Time:  17423.171  seconds.
Age:  3 . Time:  18831.0179  seconds.
Age:  2 . Time:  21746.0866  seconds.
Age:  1 . Time:  25657.8417  seconds.
TOTAL ELAPSED TIME:  25657.8423  seconds. 



## Grid Creation 

In [7]:
# Function to construct the grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for i in range(0,nx):
	xgrid[i] = xmin + it*xstep;
	it = it+1;


# Function to construct the grid for productivity (e) using Tauchen (1986)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for i in range(0,ne):
	egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
	it = it+1;


# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
	for k in range(0,ne):
		if (k == 0):
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
		elif (k == ne-1):
			P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
		else:
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);

# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]);

## Structure and Function 

In [8]:
# Value function
VV = math.pow(-10.0, 5);

specs = OrderedDict()
specs['ind'] = int64
specs['ne'] = int64
specs['nx'] = int64
specs['T'] = int64
specs['age'] = int64
specs['P'] = float64[:,:]
specs['xgrid'] = float64[:]
specs['egrid'] = float64[:]
specs['ssigma'] = float64
specs['bbeta'] = float64
specs['w'] = float64
specs['r'] = float64
specs['V'] = float64[:,:,:]


# Data structure of state and exogenous variables
@jitclass(specs)
class modelState(object):
	def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r,V):
		self.ind		= ind
		self.ne			= ne
		self.nx			= nx
		self.T			= T
		self.age		= age
		self.P			= P
		self.xgrid		= xgrid
		self.egrid		= egrid
		self.ssigma		= ssigma
		self.bbeta		= bbeta
		self.w			= w
		self.r			= r
		self.V			= V

# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix)
@njit
def value_func(states):

	ind = states.ind
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r
	V = states.V

	ix = int(math.floor(ind/ne));
	ie = int(math.floor(ind%ne));

	VV = math.pow(-10.0, 3)
	for ixp in range(0,nx):
		expected = 0.0;
		if(age < T-1):
			for iep in range(0,ne):
				expected = expected + P[ie, iep]*V[age+1, ixp, iep]

		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp];

		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;

		if(cons <= 0):
			utility = math.pow(-10.0,5);
		
		if(utility >= VV):
			VV = utility;

		utility = 0.0;

	return[VV];

## *Another Life-cycle Consumption* 

In [10]:
print(" ")
print("Life cycle computation: ")
print(" ")

@njit(parallel=True)
def compute(age, V):

	for ind in prange(0,nx*ne):

		states = modelState(ind, ne, nx, T, age, P, xgrid, egrid, ssigma, bbeta, w, r, V)
		
		ix = int(math.floor(ind/ne));
		ie = int(math.floor(ind%ne));

		V[age, ix, ie] = value_func(states)[0];

	return(V)



start = time.time()
# Initialize value function V
V     = np.zeros((T, nx, ne))

for age in range(T-1, -1, -1):
	V = compute(age, V)

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")


finish = time.time() - start
print("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")

 
Life cycle computation: 
 
Age:  10 . Time:  2.1684  seconds.
Age:  9 . Time:  2.8737  seconds.
Age:  8 . Time:  3.9767  seconds.
Age:  7 . Time:  4.8341  seconds.
Age:  6 . Time:  5.6859  seconds.
Age:  5 . Time:  6.4507  seconds.
Age:  4 . Time:  7.1632  seconds.
Age:  3 . Time:  7.8539  seconds.
Age:  2 . Time:  8.5451  seconds.
Age:  1 . Time:  9.2503  seconds.
TOTAL ELAPSED TIME:  9.2511  seconds. 



## Chekcs 

In [11]:
print(" - - - - - - - - - - - - - - - - - - - - - \n")
print("The first entries of the value function: \n")

for i in range(0,3): 
    print(round(V[0, 0, i], 5))

print(" \n")

 - - - - - - - - - - - - - - - - - - - - - 

The first entries of the value function: 

-2.11762
-2.07729
-2.02366
 



The end. 